# Add crossings

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import set_path

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

import shapely.geometry as sg
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely import wkt
import momepy
import networkx as nx
from scipy.spatial.distance import cdist
from branca.element import Template, MacroElement
from tqdm.notebook import tqdm
import branca.colormap as cm

import folium

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

In [ ]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

### Import walking and road network 

### NOTE: If the cf.output_road_network file does not exist yet, run the create_road_network.ipynb notebook before running this notebook

In [ ]:
# Get basic pedestrian network with widths
CRS = 'EPSG:28992'

gdf_network = gpd.read_file(cf.output_file_widths).to_crs(crs=CRS)
graph_network = momepy.gdf_to_nx(gdf_network, approach='primal')
cc_list = [x for y in zip(gdf_network['sidewalk_id'], gdf_network['sidewalk_id']) for x in y]
gdf_network_nodes = gpd.GeoDataFrame(geometry = gdf_network.boundary.explode(index_parts=True), crs=CRS)
gdf_network_nodes['x'], gdf_network_nodes['y'] = gdf_network_nodes.geometry.x, gdf_network_nodes.geometry.y
gdf_network_nodes['cc'] = cc_list

gdf_road_network = gpd.read_file(cf.output_road_network).to_crs(crs=CRS)

# 1. Create crossings with curb heights

### Import curb segments

In [ ]:
# Import curb heights
base_folder = cf.in_folder + 'route_planning/curb_heights/'
ch_crossing_features = cf.in_folder + 'route_planning/curb_heights/curbs_and_heights.csv'
df_ch = pd.read_csv(ch_crossing_features)
df_ch['geometry'] = df_ch['line_segm'].apply(wkt.loads)
gdf_ch = gpd.GeoDataFrame(df_ch, crs=CRS)
graph_ch = momepy.gdf_to_nx(gdf_ch, approach='primal')

### Select curb segments to keep


In [ ]:
# Create network and crossing feature dataframes

# TODO 
# Enable setting max_height >= 10cm (e.g., by generating compute with more RAM or compute crossings in smaller areas)

# SET THIS VALUE
max_height = 0.04 # in meter

gdf_ch.dropna(subset=['curb_height'])
gdf_ch = gdf_ch.loc[(gdf_ch['curb_height'] <= max_height)]
gdf_ch['centroid'] = gdf_ch['geometry'].centroid
gdf_ch.reset_index(drop=True, inplace=True)

### Determine connected components of curb edges 
#### These are used as a restrictive rule when creating possible crossing edges (i.e., crosing edges should connect different curbs)

In [ ]:
# Calculate connected components for each edge
# Option 1: From curb height edges
# Option 2: From sidewalk edges
# Option 1 results in more connected components since the curb height edges are more fragmentated (i.e., not all parts of sidewalk egde have height information)

# Calculate using the connected components retrieved from the curb height edges
cc_sidewalks = list(nx.connected_components(graph_ch))
nodes_in_edge = [line.coords[0] for line in gdf_ch['geometry'].to_list()]
cc_ids = [i for node in nodes_in_edge for i in range(len(cc_sidewalks)) if node in cc_sidewalks[i]]
gdf_ch['cc_from_curb_edges'] = cc_ids

# Calculate using the connected components retrieved from sidewalk edges from bgt (from road_curb_segments.ipynb notebook)
gdf_ch['cc_from_sidewalk_edges'] = gdf_ch.groupby('overarching_line_segm').ngroup()

### Initiate function to connect nodes in the network

In [ ]:
# Function to find closest target nodes from source nodes and subsequently create edge between tese nodes.
# Edge is a connection between a source and a target node.
# max_dist: maximmum distance between source and target node is max_dist.
# max_connections: maximum number of new edges from source node.
def get_crossing_edges_from_curb_heights(gdf_source_nodes, gdf_target_nodes, gdf_roads, min_dist=10, max_dist=20, max_connections=3, CRS='EPSG:28992'):

    # Get distance matrices
    print('Determining target nodes within distance {}m-{}m from source node...'.format(min_dist, max_dist))
    dist_matrix = cdist(gdf_source_nodes[['x', 'y']].values, gdf_target_nodes[['x', 'y']].values, metric='euclidean')
    np.fill_diagonal(dist_matrix, 10**6)
    dist_sort = np.sort(dist_matrix, axis=1)
    dist_argsort = np.argsort(dist_matrix, axis=1)
    idxs = np.where((dist_sort > min_dist) & (dist_sort < max_dist), True, False)
    nodes_to_connect = [list(row[row_idxs]) for row, row_idxs in zip(dist_argsort, idxs)]
    edges, edges_dict = [], {}
    print('Possible target nodes determined.')

    # Calculate possible crossing edges
    print('Determining possible edges from source nodes to target nodes...')
    edges, edges_dict = [], {}
    for source_node in tqdm(range(len(dist_sort))):
        source_cc = gdf_source_nodes.loc[source_node, 'cc_from_sidewalk_edges']
        restricted_cc, connections_count = [source_cc], 0
        if source_cc not in edges_dict.keys():
            edges_dict[source_cc] = {}

        # Loop over possible target nodes for source node
        for target_node in nodes_to_connect[source_node]:
            target_cc = gdf_target_nodes.loc[target_node, 'cc_from_sidewalk_edges']

            # Check if target node is not part of restricted connected component
            if target_cc not in restricted_cc and connections_count < max_connections:
                
                pos_edge = sg.LineString([gdf_source_nodes.iloc[source_node]['centroid'], gdf_target_nodes.iloc[target_node]['centroid']])
                pos_length = pos_edge.length

                # Add crossing edge if distance between source connected component and target node is the shortest known route
                if target_node in edges_dict[source_cc].keys():
                    if edges_dict[source_cc][target_node]['length'] > pos_length:
                        edges_dict[source_cc][target_node]['edge'] = pos_edge
                        edges_dict[source_cc][target_node]['length'] = pos_length
                        connections_count += 1
                        restricted_cc.append(target_cc)
                else:
                    edges_dict[source_cc][target_node] = {}
                    edges_dict[source_cc][target_node]['edge'] = pos_edge
                    edges_dict[source_cc][target_node]['length'] = pos_length

                    connections_count += 1
                    restricted_cc.append(target_cc)

    print('Determined possible edges from source nodes to target nodes.')

    print('Removing edges that do not cross the street or bikepath...')
    edges_geometries = [edges_dict[source_cc][target_node].get('edge') for source_cc in edges_dict.keys() for target_node in edges_dict[source_cc].keys()]
    edges_geometries_final = [edge for edge in edges_geometries if gdf_roads.intersects(edge).nunique() > 1]
    print('Removed edges that do not cross the street or bikepath.')

    gdf_edges = gpd.GeoDataFrame(geometry=edges_geometries_final, crs=CRS)
    return gdf_edges

### Calculate curb ramp crosswalks 

In [ ]:
# TODO
# Include angle of intersection between crosswalk edge and road centerline as critera

# Get coordinates of crossing feature nodes
gdf_ch['geometry'] = gdf_ch['centroid']
gdf_ch['x'], gdf_ch['y'] = gdf_ch.geometry.x, gdf_ch.geometry.y

# Connect curb height nodes to curb height nodes
min_dist = 0
max_dist = 11
max_connections = 1
gdf_curb_edges = get_crossing_edges_from_curb_heights(gdf_ch, gdf_ch, gdf_road_network, 
                    min_dist=min_dist, max_dist=max_dist, max_connections=max_connections)
print('Number of new edges from curb node to curb node:', len(gdf_curb_edges))

### Connect curb ramp crossings to walking network

In [ ]:
# TODO remove the connected component stuff (because not needed)

def connect_crossing_edge(gdf_source_nodes, gdf_target_nodes, walking_graph, max_dist=20, max_connections=3, CRS='EPSG:28992', network_to_network=False):

    # Get distance matrices
    dist_matrix = cdist(gdf_source_nodes[['x', 'y']].values, gdf_target_nodes[['x', 'y']].values, metric='euclidean')
    np.fill_diagonal(dist_matrix, 10**6)
    dist_sort = np.sort(dist_matrix, axis=1)
    dist_argsort = np.argsort(dist_matrix, axis=1)

    # Calculate new edges
    idxs = np.where(dist_sort < max_dist, True, False)
    nodes_to_connect = [list(row[row_idxs]) for row, row_idxs in zip(dist_argsort, idxs)]
    edges = [[i, nodes_to_connect[i][j]] for i in range(len(dist_sort)) for j in range(len(nodes_to_connect[i])) if j < max_connections]
    if len(edges) == 2:

        # Only return connecting edges if crossing edge is connected to different sidewalks
        target_cc = [gdf_target_nodes.iloc[edge[1]]['cc'] for edge in edges]

        # check length between network target nodes
        try:
            point_1_network = gdf_target_nodes.iloc[edges[0][1]]['geometry']
            point_2_network = gdf_target_nodes.iloc[edges[1][1]]['geometry']
            node_1_network = (point_1_network.x, point_1_network.y)
            node_2_network = (point_2_network.x, point_2_network.y)
            sp_length = nx.shortest_path_length(walking_graph, node_1_network, node_2_network)
        except:
            sp_length = 1000

        if len(set(target_cc)) == 2 or (len(set(target_cc)) == 1 and sp_length > 20):
            if network_to_network:
                edges_geometries = [sg.LineString([gdf_target_nodes.iloc[edges[0][1]]['geometry'], gdf_target_nodes.iloc[edges[1][1]]['geometry']])]
            else:
                edges_geometries = [sg.LineString([gdf_source_nodes.iloc[edge[0]]['geometry'], gdf_target_nodes.iloc[edge[1]]['geometry']]) for edge in edges]
                edges_geometries.append(sg.LineString([gdf_source_nodes.iloc[0]['geometry'], gdf_source_nodes.iloc[1]['geometry']]))
            gdf_edges = gpd.GeoDataFrame(geometry=edges_geometries, crs=CRS)
            return gdf_edges
        else:
            return gpd.GeoDataFrame()

In [ ]:
# Current heuristic: Connect outer points of crossing edge to closest network nodes within max_dis.
# Network nodes cannot be part of the same sidewalk
# TODO make this neater

# Get outer nodes of crossing edges and connect to network
gdf_curb_edge_nodes = gpd.GeoDataFrame(geometry=gdf_curb_edges['geometry'].boundary.explode(index_parts=True), crs=CRS).reset_index()
gdf_curb_edge_nodes['x'], gdf_curb_edge_nodes['y'] = gdf_curb_edge_nodes.geometry.x, gdf_curb_edge_nodes.geometry.y
groups = gdf_curb_edge_nodes.groupby(np.arange(len(gdf_curb_edge_nodes.index))//2)

# Loop over possible crossing edges seperately and connect to walking network
gdf_crossings = gpd.GeoDataFrame()
network_to_network = True # Set False if you wish to connect network to curb instead of network to network 

for (idx, frame) in tqdm(groups):
    sub_gdf = connect_crossing_edge(frame, gdf_network_nodes, graph_network, max_dist=4, max_connections=1, network_to_network=network_to_network)
    gdf_crossings = pd.concat([gdf_crossings, sub_gdf])

print('Connected valid crossings')

### create map and visualize crossings on map

In [ ]:
# Initialize legend
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:black;opacity:0.7;'></span>Network Edge </li>
    <li><span style='background:red;opacity:0.7;'></span>Crossing Edge</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

In [ ]:
# Create tooltip for feature representation on map
def gen_tooltip(fields, aliases):

    tooltip = folium.GeoJsonTooltip(
        fields=fields,
        aliases=aliases,
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
    return tooltip

In [ ]:
# set True for satelite background, False for regular background
satelite = False

# Set folium map background
if satelite == True:
    network_color = 'white'
    tile = folium.TileLayer(
                tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                attr = 'Esri',
                name = 'Esri Satellite',
                overlay = False,
                control = True)
else:
    tile = 'openstreetmap'
    network_color = 'black'

# Create Folium map
map = folium.Map(
    location=[52.389164, 4.908453], tiles=tile,
    min_zoom=10, max_zoom=25, zoom_start=15,
    zoom_control=True, control_scale=True, control=False
    )

# Add colormap
cmp = cm.linear.RdYlGn_11.colors
cmp = list(reversed(cmp))
colormap = cm.LinearColormap(colors=cmp, vmin=0, vmax=0.2, caption='Curb height (m)')
colormap.add_to(map)

# Add network and new edges
geo_j = folium.GeoJson(gdf_network, style_function=lambda x: {"color": "black", "weight": 2}).add_to(map)
geo_j = folium.GeoJson(gdf_crossings, style_function=lambda x: {"color": "red", "weight": 4}).add_to(map)


# # Add curb heights
# gdf_ch['geometry'] = gdf_ch['line_segm'].astype(str).apply(wkt.loads)
# gdf_ch = gdf_ch.drop(columns=['line_segm', 'centroid', 'line_segm_polygon', 'overarching_line_segm'])
# json = gdf_ch.to_crs(crs='EPSG:4326').to_json()
# folium.GeoJson(json, style_function=lambda feature: {"color": colormap(feature["properties"]['curb_height']) if feature['properties']['curb_height'] != 0 else 'black', 
#             "weight": 4 if feature["properties"]['curb_height'] != 0 else 2}).add_to(map)

# Save map
map.get_root().add_child(macro)
map.save(cf.out_folder + 'extracted_crossings/curb_crossings_max_height_{}_min_dist_{}_max_dist_{}_max_conn_{}.html'.format(max_height, min_dist, max_dist, max_connections))

### Save new edges

In [ ]:
gdf_crossings.to_file(cf.output_curb_crossings_base + '_max_height_{}.gpkg'.format(max_height), driver='GPKG')